In [1]:
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import torch

c:\Universidad\Legislacion\Scrapy threads\Analisis de sentimientos\sentiment_analysis\sentiment\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset

In [4]:
dataset = load_dataset('tweet_eval', 'sentiment')

Generating validation split: 100%|██████████| 2000/2000 [00:00<?, ? examples/s]


In [10]:
for i in dataset:
    print(i)

train
test
validation


In [2]:
random_seed = 0
max_len = 200
batch_size = 16
dataset_path = 'C:/Users/andre/Downloads/datos_threads.csv'
n_classes = 2

In [3]:
np.random.seed(random_seed)
torch.manual_seed(random_seed)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
df = pd.read_csv(dataset_path, sep=';')
print(df)

                                             post_link user_name  \
0    https://www.threads.net/@farandulaalrojovivo/p...      Hilo   
1    https://www.threads.net/@achirasgiron/post/C9f...      Hilo   
2    https://www.threads.net/@el.azuayo/post/C9Km1W...      Hilo   
3    https://www.threads.net/@ecuainm1/post/C9KjVbf...      Hilo   
4    https://www.threads.net/@jaircarrion_/post/C9B...      Hilo   
..                                                 ...       ...   
295  https://www.threads.net/@pajaropolitico/post/C...      Hilo   
296  https://www.threads.net/@radiodinamicaecu/post...      Hilo   
297  https://www.threads.net/@radiodinamicaecu/post...      Hilo   
298  https://www.threads.net/@mmcuesta/post/C1_935I...      Hilo   
299  https://www.threads.net/@tctelevision/post/C0M...      Hilo   

                                     profile_link                 post_date  \
0    https://www.threads.net/@farandulaalrojovivo  2024-07-17T00:03:48.000Z   
1           https://www.t

In [5]:
pre_trained_model = 'dccuchile/bert-base-spanish-wwm-cased'
tokenizer = BertTokenizer.from_pretrained(pre_trained_model)

In [6]:
sample_txt = 'Estoy comiendo en familia'
tokens = tokenizer.tokenize(sample_txt)
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
print('Frase: ', sample_txt)
print('Tokens: ', tokens)
print('Tokens numéricos: ', tokens_ids)

Frase:  Estoy comiendo en familia
Tokens:  ['Estoy', 'comiendo', 'en', 'familia']
Tokens numéricos:  [2040, 15139, 1036, 2268]


In [7]:
encoding = tokenizer.encode_plus(
    sample_txt,
    max_length=10,
    truncation=True,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt'
)


c:\Universidad\Legislacion\Scrapy threads\Analisis de sentimientos\sentiment_analysis\sentiment\Lib\site-packages\transformers\tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
